In [1]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPool2D,Flatten , BatchNormalization
from tensorflow.keras.models import Sequential
import pandas as pd 
import numpy as np

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
x_train = train.drop(["label"], axis=1) 
y_train = train["label"].values

In [4]:
x_train = (x_train.astype("float32")/255.0).to_numpy().reshape(-1, 28,28, 1)
test = (test.astype("float32")/255.0).to_numpy().reshape(-1,28,28,1)

In [5]:
y_train = keras.utils.to_categorical(y_train)

In [6]:

model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))

In [10]:
optim = keras.optimizers.RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay=0.0)
model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        3

In [12]:
model.fit(x_train,y_train, epochs=21)

Train on 42000 samples
Epoch 1/21
42000/42000 [==============================] - 22s 520us/sample - loss: 0.1026 - accuracy: 0.9685
Epoch 2/21
42000/42000 [==============================] - 21s 494us/sample - loss: 0.0518 - accuracy: 0.9839
Epoch 3/21
42000/42000 [==============================] - 21s 496us/sample - loss: 0.0408 - accuracy: 0.9880
Epoch 4/21
42000/42000 [==============================] - 21s 510us/sample - loss: 0.0338 - accuracy: 0.9902
Epoch 5/21
42000/42000 [==============================] - 23s 539us/sample - loss: 0.0279 - accuracy: 0.9920
Epoch 6/21
42000/42000 [==============================] - 23s 544us/sample - loss: 0.0262 - accuracy: 0.9924
Epoch 7/21
42000/42000 [==============================] - 21s 507us/sample - loss: 0.0236 - accuracy: 0.9930
Epoch 8/21
42000/42000 [==============================] - 22s 518us/sample - loss: 0.0231 - accuracy: 0.9933
Epoch 9/21
42000/42000 [==============================] - 21s 501us/sample - loss: 0.0215 - accuracy: 0.9

In [13]:
preds = model.predict(test)
submission = np.argmax(preds, axis=1)
submission = pd.DataFrame({"ImageId": range(1, 28001), "Label":submission})

In [14]:
submission.to_csv("submission-keras.csv", index=False)